Take April and as City New York City

In [1]:
import pandas as pd
import numpy as np
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import lars_path
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.metrics import r2_score
import scipy.stats as stats
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [2]:
airbnbAPR= pd.read_csv("/Users/galamboslajos/Desktop/Amsterdam_HW2/listings 2.csv")
airbnbAPR.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,761411,https://www.airbnb.com/rooms/761411,20230903194145,2023-09-03,city scrape,Condo in Amsterdam · ★4.74 · 1 bedroom · 1 bed...,"Really quiet, spacious and safe, a nice place ...","The neighborhood is very green, quiet, safe an...",https://a0.muscache.com/pictures/10591508/bf30...,4013546,...,4.92,4.42,4.65,0363 D4AD DCF3 E72A 56A2,t,2,0,2,0,2.30
1,768274,https://www.airbnb.com/rooms/768274,20230903194145,2023-09-04,city scrape,Rental unit in Amsterdam · ★4.83 · 1 bedroom ·...,Our cool and spacious loft is perfect for a st...,Our neighbourhood is ons of the most beautiful...,https://a0.muscache.com/pictures/27206847/31f5...,3678216,...,4.94,4.86,4.77,0363 7A50 18E7 51D1 B7F9,t,1,1,0,0,0.64
2,768737,https://www.airbnb.com/rooms/768737,20230903194145,2023-09-03,city scrape,Boat in Amsterdam · ★4.82 · 1 bedroom · 1 bed ...,Room to rent in my houseboat. The room has a p...,It is just outside the Jordan in between the c...,https://a0.muscache.com/pictures/1df40445-354f...,3877342,...,4.87,4.72,4.68,036396BE30827DDB9575,t,3,1,2,0,2.73
3,771217,https://www.airbnb.com/rooms/771217,20230903194145,2023-09-04,previous scrape,Houseboat in Amsterdam · ★5.0 · 3 bedrooms · 3...,"Spacious houseboat in Amsterdam, suitable for ...",The houseboat lies in an area with ± 200 house...,https://a0.muscache.com/pictures/57715927/0808...,4068486,...,5.00,4.67,4.56,0363 D807 AD6C 499A F871,f,1,1,0,0,0.11
4,771343,https://www.airbnb.com/rooms/771343,20230903194145,2023-09-03,city scrape,Rental unit in Amsterdam · ★4.89 · 1 bedroom ·...,Royal Bed & Coffee Room with a very comfortabl...,The building is located in Amsterdam centre in...,https://a0.muscache.com/pictures/ea22d262-7456...,2313061,...,4.91,4.96,4.79,0363 8C61 E9B9 5582 913E,f,2,0,2,0,6.62


In [3]:
column_info = pd.DataFrame({
    "ID": range(len(airbnbAPR.columns)),
    "Variable": airbnbAPR.columns
})
print(column_info)

    ID                                      Variable
0    0                                            id
1    1                                   listing_url
2    2                                     scrape_id
3    3                                  last_scraped
4    4                                        source
..  ..                                           ...
70  70                calculated_host_listings_count
71  71   calculated_host_listings_count_entire_homes
72  72  calculated_host_listings_count_private_rooms
73  73   calculated_host_listings_count_shared_rooms
74  74                             reviews_per_month

[75 rows x 2 columns]


### Cleaning Dataset
#### After indexing the columns, i dropped some of them (can be checked under). Moreover, I decided to keep a threshold of 40% for varibales with NAs. If we are too narrow, we will reduce the number of variable drastically.

In [4]:
columns_to_drop = [
    0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
    11, 12, 13, 15, 20,
    21,
    44, 45, 46, 47,
    56, 60,
    61, 69
]
airbnb_dropped = airbnbAPR.drop(columns=airbnbAPR.columns[columns_to_drop])
threshold = 0.4
airbnb_nmiss = airbnb_dropped.loc[:, airbnb_dropped.isnull().mean() < threshold]
print(airbnb_nmiss.describe())
airbnb2 = airbnb_nmiss.copy()
airbnb2['price'] = airbnb2['price'].replace('[\$,]', '', regex=True).astype(float)

airbnb2['price'] = pd.to_numeric(airbnb2['price'], errors='coerce')
airbnb2 = airbnb2[airbnb2['price'] <= 10000]
airbnb2['ln_price'] = np.log(airbnb2['price'])

       host_listings_count  host_total_listings_count     latitude  \
count          8386.000000                8386.000000  8386.000000   
mean              3.488433                   6.217625    52.366603   
std              26.562565                  57.633911     0.017191   
min               1.000000                   1.000000    52.290340   
25%               1.000000                   1.000000    52.355640   
50%               1.000000                   1.000000    52.365769   
75%               1.000000                   3.000000    52.376440   
max             672.000000                1555.000000    52.425159   

         longitude  accommodates     bedrooms         beds  minimum_nights  \
count  8386.000000   8386.000000  6958.000000  8313.000000     8386.000000   
mean      4.889834      2.901741     1.664128     1.831348        5.046625   
std       0.035403      1.328242     0.940524     1.441244       34.710395   
min       4.755870      1.000000     1.000000     1.00000

In [5]:
for column in airbnb2.columns:
    value_type = type(airbnb2.loc[0, column])
    print(f"Column '{column}' has values of type: {value_type}")

Column 'host_response_rate' has values of type: <class 'str'>
Column 'host_acceptance_rate' has values of type: <class 'str'>
Column 'host_is_superhost' has values of type: <class 'str'>
Column 'host_thumbnail_url' has values of type: <class 'str'>
Column 'host_listings_count' has values of type: <class 'numpy.int64'>
Column 'host_total_listings_count' has values of type: <class 'numpy.int64'>
Column 'host_verifications' has values of type: <class 'str'>
Column 'host_has_profile_pic' has values of type: <class 'str'>
Column 'host_identity_verified' has values of type: <class 'str'>
Column 'neighbourhood' has values of type: <class 'str'>
Column 'neighbourhood_cleansed' has values of type: <class 'str'>
Column 'latitude' has values of type: <class 'numpy.float64'>
Column 'longitude' has values of type: <class 'numpy.float64'>
Column 'property_type' has values of type: <class 'str'>
Column 'room_type' has values of type: <class 'str'>
Column 'accommodates' has values of type: <class 'num

In [6]:
airbnb2.drop(columns=['room_type'], inplace=True)
airbnb2.drop(columns=['host_thumbnail_url'], inplace=True)
print(airbnb2['review_scores_value'].value_counts)

<bound method IndexOpsMixin.value_counts of 0       4.65
1       4.77
2       4.68
3       4.56
4       4.79
        ... 
8381     NaN
8382     NaN
8383     NaN
8384     NaN
8385     NaN
Name: review_scores_value, Length: 8384, dtype: float64>


Regarding the property type: we merged observations which falls into the category of Private Room, House and Shared Room. Then all the other observations have been dropped.

In [7]:
airbnb2['property_type'] = airbnb2['property_type'].apply(lambda x: 'Private room' if x.startswith('Private room') else x)
airbnb2['property_type'] = airbnb2['property_type'].apply(lambda x: 'Private room' if x.startswith('Room') else x)
airbnb2['property_type'] = airbnb2['property_type'].apply(lambda x: 'House' if x.startswith('Entire') else x)
airbnb2['property_type'] = airbnb2['property_type'].apply(lambda x: 'Shared Room' if x.startswith('Shared') else x)
property_type_counts = airbnb2['property_type'].value_counts().reset_index()
property_type_counts.columns = ['Property_Type', 'Count']
column_info2 = pd.DataFrame({
    "ID": range(len(airbnbAPR.property_type)),
    "Variable": airbnbAPR.property_type
})
column_info2

allowed_types = ['Private room', 'House', 'Shared Room']
airbnb2 = airbnb2[airbnb2['property_type'].isin(allowed_types)]
property_type_counts

,Property_Type,Count
0,House,6187
1,Private room,1878
2,Houseboat,155
3,Boat,101
4,Shared Room,42
5,Tiny home,10
6,Tent,3
7,Camper/RV,2
8,Casa particular,2
9,Barn,1


In [8]:
property_dummies = pd.get_dummies(airbnb2['property_type'], prefix='property', drop_first=False)
airbnb2 = pd.concat([airbnb2, property_dummies], axis=1)
airbnb2

,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,...,license,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,ln_price,property_House,property_Private room,property_Shared Room
0,100%,100%,t,2,2,"['email', 'phone']",t,t,"Amsterdam, North Holland, Netherlands",Noord-Oost,...,0363 D4AD DCF3 E72A 56A2,2,0,2,0,2.30,4.110874,False,True,False
1,100%,100%,f,1,4,"['email', 'phone']",t,t,"Amsterdam, North Holland, Netherlands",Westerpark,...,0363 7A50 18E7 51D1 B7F9,1,1,0,0,0.64,5.789960,True,False,False
2,100%,100%,t,3,3,"['email', 'phone', 'work_email']",t,t,"Amsterdam, North Holland, Netherlands",Westerpark,...,036396BE30827DDB9575,3,1,2,0,2.73,4.691348,False,True,False
4,100%,97%,t,2,2,"['email', 'phone']",t,t,"Amsterdam, Noord-Holland, Netherlands",Centrum-West,...,0363 8C61 E9B9 5582 913E,2,0,2,0,6.62,5.010635,False,True,False
5,100%,100%,t,2,2,"['email', 'phone']",t,t,"Amsterdam, North Holland, Netherlands",Slotervaart,...,0363 BCBA 6311 3268 DC8C,2,0,2,0,2.20,4.317488,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8381,NaN,NaN,f,1,3,"['email', 'phone']",t,t,NaN,Oud-Noord,...,0363 9080 FE38 A7CE E7B7,1,1,0,0,NaN,5.521461,True,False,False
8382,NaN,NaN,f,1,1,"['email', 'phone']",t,t,NaN,Oud-Noord,...,0363 5639 0AD2 7A88 3FED,1,1,0,0,NaN,5.820083,True,False,False
8383,100%,83%,f,1,1,"['email', 'phone']",t,t,"Amsterdam, Noord-Holland, Netherlands",Bos en Lommer,...,0363 C287 34D9 EA25 9428,1,1,0,0,NaN,4.905275,True,False,False
8384,100%,100%,f,2,3,"['email', 'phone']",t,t,NaN,Centrum-West,...,Exempt,1,0,1,0,NaN,5.075174,False,True,False


### Dealing with Amenities

In [9]:
print(airbnb2['amenities'].head(1))

amenities = airbnb2['amenities'].str.split(',')
amenities_table = pd.Series([item for sublist in amenities for item in sublist]).value_counts()
sorted_table = amenities_table.sort_values(ascending=False)
print(sorted_table.head())

0    ["Extra pillows and blankets", "Drying rack fo...
Name: amenities, dtype: object
 "Smoke alarm"              7279
 "Essentials"               7064
 "Wifi"                     6592
 "Dishes and silverware"    6227
 "Hair dryer"               5925
Name: count, dtype: int64


### Transform amenities into a binary indicator matrix

In [10]:
amenities_table = airbnb2['amenities'].str.get_dummies(sep=',')
amenities_table

,""" Boretti stainless steel gas stove""",""" Boretti stainless steel gas stove""",""" Boretti stainless steel oven""","""* Use 'Liebherr' fridge in kitchen. * I wash the dishes. refrigerator""","""- body soap""","""- conditioner""","""- refrigerator""","""- shampoo""",""". body soap""",""". body soap""",...,"[""Spa Collection body soap""","[""Stove""","[""Syoss shampoo""","[""TV""","[""TV""]","[""Trash compactor""","[""Washer""","[""Waterfront""","[""Wine glasses""",[]
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8381,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8382,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8383,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8384,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Top 10 Amenities in Amsterdam

In [11]:
amenities_nyc = airbnb2['amenities'].str.split(',')
amenities_table_nyc = pd.Series([item for sublist in amenities_nyc for item in sublist]).value_counts()

# Top 10 amenities in NYC
top_10_nyc_amenities = amenities_table_nyc.head(10)
print(top_10_nyc_amenities)

 "Smoke alarm"              7279
 "Essentials"               7064
 "Wifi"                     6592
 "Dishes and silverware"    6227
 "Hair dryer"               5925
 "Hangers"                  5714
 "Iron"                     5618
 "Hot water"                5465
 "Bed linens"               5463
 "Cooking basics"           5141
Name: count, dtype: int64


In [12]:
amenities = [col for col in airbnb2 if col.startswith("d_")]

In [13]:
variable_names = airbnb2.columns.tolist()
variable_table = pd.DataFrame(variable_names, columns=["Variable Names"])
pd.set_option('display.max_rows', None)
print(variable_table)

                                  Variable Names
0                             host_response_rate
1                           host_acceptance_rate
2                              host_is_superhost
3                            host_listings_count
4                      host_total_listings_count
5                             host_verifications
6                           host_has_profile_pic
7                         host_identity_verified
8                                  neighbourhood
9                         neighbourhood_cleansed
10                                      latitude
11                                     longitude
12                                 property_type
13                                  accommodates
14                                bathrooms_text
15                                      bedrooms
16                                          beds
17                                     amenities
18                                         price
19                  

In [14]:
airbnb2.to_csv('/Users/galamboslajos/Desktop/Amsterdam_HW2/listings_amsterdam_FINAL.csv', index=False)